Imports

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

Pour analyser les données, afin de ne pas être obligés à chaque ouverture de run tout le code, on utilise directement le fichier csv créé.

In [6]:
final_data = pd.read_csv("Base finale.csv")

Partie 1: Statistiques descriptives

A) Analyse des prix littéraires

In [8]:
prix_counts = final_data['prix_19_23'].value_counts()
prix_proportions = final_data['prix_19_23'].value_counts(normalize=True) * 100

print("Nombre de livres ayant reçu un prix littéraire :")
print(prix_counts)
print("\nProportion de livres ayant reçu un prix littéraire (%):")
print(prix_proportions)

Nombre de livres ayant reçu un prix littéraire :
prix_19_23
1.0    258
0.0    233
Name: count, dtype: int64

Proportion de livres ayant reçu un prix littéraire (%):
prix_19_23
1.0    52.545825
0.0    47.454175
Name: proportion, dtype: float64


Parmi tous les livres de la base, étant les livres les plus lus, les mieux notés, et les plus récompensés, on peut être surpris que les livres récompensés ne représentent qu'à peu près la moitié des livres de la base, surtout que c'est de la base Wikipedia que l'on a récupéré le plus de livres.
Il est encore plus intéressant de regarder combien de livres étant aussi dans une autre base que wikipedia ont eu un prix. 

In [9]:
# on modifie la table pour créer une fonction qui renvoie 1 si le livre est dans la table wikipedia (ie a un prix)
# ET dans une autre table

def wiki_plus(row):
    if row['prix_19_23'] == 1 and ((row['top_fnac_1'] == 1) or (row['top_babelio'] == 1) or (row['top_livraddict'] == 1) or (row['Top bibliothèque'] == 1)):
        plus_wiki = 1
    else:
        plus_wiki = 0
    return plus_wiki

In [10]:
final_data['Prix_et_plus'] = final_data.apply(wiki_plus, axis=1)

In [ ]:
# Visualisation des prix littéraires
prix_counts.plot(kind='bar', title='Livres avec ou sans prix littéraire', ylabel='Nombre de livres', xlabel='Prix littéraire (1=oui, 0=non)')
plt.show()

# ------------ Analyse des tops -------------
print("\n--- Analyse des tops ---")

# Comptage et proportions pour chaque top
tops = ['top_fnac_1', 'top_fnac_2_plus', 'top_babelio', 'top_livraddict', 'any_top_indicator']
for top in tops:
    print(f"\nAnalyse pour {top}:")
    counts = final_data[top].value_counts()
    proportions = final_data[top].value_counts(normalize=True) * 100
    print(f"Nombre de livres :\n{counts}")
    print(f"Proportions (%):\n{proportions}")

"""
Sur les 229 livres
78 des livres sont dans au moins un top fnac du mois
28 sont dans 2 top fnac ou plus (on eté plusieurs mois d'affilé dans le top fnac)
25 sont dans le top babelio 2023
85 sont dans le top livraddict 2023

59 sont dans au moins un des top fnac, babelio ou livraddict
"""

# Visualisation
title = "nombre de livres par top"
counts.plot(kind='bar', title=f'Présence dans {top}', ylabel='Nombre de livres', xlabel=f'{top} (1=oui, 0=non)')
plt.show()

# ------------ Prix et présence dans les tops -------------
"""
on croise prix_19_20 avec any_top_indicator 
pour afficher la proportion de livres primés présents ou non dans les tops
"""

print("\n--- Prix et présence dans les tops ---")

prix_et_any_top = pd.crosstab(final_data['prix_19_20'], final_data['any_top_indicator'], normalize='index') * 100

print("Proportion de livres primés présents ou non dans les tops (%):")
print(prix_et_any_top)

prix_et_any_top.plot(kind='bar', stacked=True, title='Prix littéraire et présence dans les tops', ylabel='Proportion (%)', xlabel='Prix littéraire (1=oui, 0=non)')
plt.show()

"""
Parmi les livres primés (prix_19_20 = 1), seulement 7.81% sont 
présents dans un top, tandis que 92.19% ne figurent dans aucun top. 
Cela suggère que les livres primés ne sont pas nécessairement plus représentés 
dans les tops par rapport aux livres non primés.
"""

# ------------ Fréquence des auteurs -------------
print("\n--- Fréquence des auteurs ---")

auteur_counts = final_data['Auteur'].value_counts()

print("Nombre de livres par auteur :")
print(auteur_counts)

auteur_counts.head(10).plot(kind='bar', title='Top 10 des auteurs par nombre de livres', ylabel='Nombre de livres', xlabel='Auteur')
plt.show()

"""
Les auteurs Sarah Rivens et Mr Tan se démarquent avec 4 livres chacun, 
suivis par Sibylle Grimbert et Neige Sinno avec 3 livres. Cependant, la majorité des 204 auteurs 
n'ont écrit qu'un seul livre, illustrant une forte diversité mais une faible répétition dans 
les contributions des auteurs.
"""

# ------------ Livres présents dans plusieurs tops -------------
print("\n--- Livres présents dans plusieurs tops ---")
tops_columns = ['top_fnac_1', 'top_babelio', 'top_livraddict']
final_data['multiple_tops'] = (final_data[tops_columns].sum(axis=1) >= 2).astype(int)
multiple_tops_counts = final_data['multiple_tops'].value_counts()
multiple_tops_proportions = final_data['multiple_tops'].value_counts(normalize=True) * 100

print("Nombre de livres présents dans au moins deux tops :")
print(multiple_tops_counts)
print("\nProportion de livres présents dans au moins deux tops (%):")
print(multiple_tops_proportions)

multiple_tops_counts.plot(kind='bar', title='Livres présents dans au moins deux tops', ylabel='Nombre de livres', xlabel='Présence dans plusieurs tops (1=oui, 0=non)')
plt.show()

"""
Seulement 17 livres (7,42%) sont présents dans au moins deux tops, 
tandis que la grande majorité (212 livres, 92,58%) n’apparaissent que 
dans un seul top ou aucun. Cela montre que peu de livres réussissent à 
se démarquer de manière significative dans plusieurs classements.
"""